In [1]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None

In [2]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.06)

In [3]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

start = time.time()
output = simulation(policy, n=200000, seed=seed)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


349.10499930381775

In [4]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1540.7609683932544, 0.007703804841966272, 184)

In [5]:
output['history']

,impressions,clicks,lifetime,p
153,18970.0,4199.0,18003.025431,0.220134
162,228.0,26.0,1537.166719,0.113780
172,163.0,32.0,19648.592394,0.219968
173,170.0,18.0,12771.474990,0.122694
180,26.0,3.0,4655.819793,0.020061
182,6.0,0.0,889.624649,0.004621
183,1.0,0.0,15187.163761,0.073886


In [2]:

class UCB:
    def __init__(self, C):
        self.t = 1
        self.C = C

    def __call__(self, history: pd.DataFrame):
        ctr = history['clicks'] / (history['impressions'] + 1)
        u = ctr + self.C * np.sqrt(2 * np.log(self.t) / (history['impressions'] + 1))
        self.t += 1
        return history.index[np.argmax(u)]

In [3]:
seed = 18475

outputs = []

for C in [0.1, 0.5, 1, 2]:
    np.random.seed(seed=seed)
    output = simulation(UCB(C), n=200000, seed=seed)
    outputs.append((C, output['regret'], output['regret']/output['rounds'], output['total_banners']))


1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impr

Попробуем еще улучшить, изменяя параметр C перед корнем

In [4]:
for o in outputs:
    print(f'C={o[0]}, regret={o[1]}, regret/rounds={o[2]}, total_banners={o[3]}')

C=0.1, regret=258.3158253051061, regret/rounds=0.0012915791265255305, total_banners=184
C=0.5, regret=4012.2130735468863, regret/rounds=0.020061065367734433, total_banners=184
C=1, regret=11128.64414178645, regret/rounds=0.055643220708932246, total_banners=184
C=2, regret=20370.61727859559, regret/rounds=0.10185308639297795, total_banners=184


In [5]:
seed = 18475

outputs = []

for C in [0.01, 0.05]:
    np.random.seed(seed=seed)
    output = simulation(UCB(C), n=200000, seed=seed)
    outputs.append((C, output['regret'], output['regret']/output['rounds'], output['total_banners']))

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impr

In [6]:
for o in outputs:
    print(f'C={o[0]}, regret={o[1]}, regret/rounds={o[2]}, total_banners={o[3]}')

C=0.01, regret=974.1013870333624, regret/rounds=0.004870506935166812, total_banners=184
C=0.05, regret=2779.496405967007, regret/rounds=0.013897482029835035, total_banners=184


In [7]:
seed = 18475

outputs = []

for C in [0.15]:
    np.random.seed(seed=seed)
    output = simulation(UCB(C), n=200000, seed=seed)
    outputs.append((C, output['regret'], output['regret']/output['rounds'], output['total_banners']))

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


In [8]:
for o in outputs:
    print(f'C={o[0]}, regret={o[1]}, regret/rounds={o[2]}, total_banners={o[3]}')

C=0.15, regret=468.1912808048057, regret/rounds=0.002340956404024028, total_banners=184


In [24]:
seed = 18475

outputs = []

for C in [0.08, 0.12]:
    np.random.seed(seed=seed)
    output = simulation(UCB(C), n=200000, seed=seed)
    outputs.append((C, output['regret'], output['regret']/output['rounds'], output['total_banners']))

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impr

In [25]:
for o in outputs:
    print(f'C={o[0]}, regret={o[1]}, regret/rounds={o[2]}, total_banners={o[3]}')

C=0.08, regret=197.72686196531987, regret/rounds=0.0009886343098265994, total_banners=184
C=0.12, regret=324.9506848229653, regret/rounds=0.0016247534241148264, total_banners=184


In [26]:
seed = 18475

outputs = []

for C in [0.06, 0.07, 0.09]:
    np.random.seed(seed=seed)
    output = simulation(UCB(C), n=200000, seed=seed)
    outputs.append((C, output['regret'], output['regret']/output['rounds'], output['total_banners']))

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impr

In [27]:
for o in outputs:
    print(f'C={o[0]}, regret={o[1]}, regret/rounds={o[2]}, total_banners={o[3]}')

C=0.06, regret=224.59694995443272, regret/rounds=0.0011229847497721637, total_banners=184
C=0.07, regret=152.6182167021829, regret/rounds=0.0007630910835109146, total_banners=184
C=0.09, regret=228.79558491386356, regret/rounds=0.0011439779245693178, total_banners=184


Лучший результат при C=0.07, оставим его

In [20]:
seed = 18475
np.random.seed(seed=seed)

output = simulation(UCB(0.07), n=200000, seed=seed)
print(f"regret={output['regret']}, regret/rounds={output['regret']/output['rounds']}, total_banners={output['total_banners']}")

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
regret=152.6182167021829, regret/rounds=0.0007630910835109146, total_banners=184


In [19]:
output['history']

,impressions,clicks,lifetime,p
153,3.0,0.0,18003.025431,0.220134
162,3.0,0.0,1537.166719,0.113780
172,21203.0,4624.0,19648.592394,0.219968
173,2.0,0.0,12771.474990,0.122694
180,2.0,0.0,4655.819793,0.020061
182,2.0,0.0,889.624649,0.004621
183,2.0,0.0,15187.163761,0.073886
